Install needed libraries

In [1]:
!pip install langchain-community faiss-cpu
!pip install sentence-transformers
!pip install streamlit
!pip install langchain_huggingface

Imports needed modules

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
import re
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4
from langchain_core.documents import Document
import streamlit as st

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

Generate model

In [3]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Generate Faiss Vector store

In [4]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

Opens the text file and cleans up the data

In [5]:
with open("blogs/blog 2.txt", "r", encoding="utf-8") as file:
    text = file.read()

sentences=[s.strip() + '.' for s in re.split(r'\.\s*', text) if s]

chunks = []
current_chunk = []
current_word_count = 0
for sentence in sentences:
       word_count = len(sentence.split())
       if current_word_count + word_count <= 200:
           current_chunk.append(sentence)
           current_word_count += word_count
       else:
           # Commit current chunk and start a new one
           chunks.append(' '.join(current_chunk))
           current_chunk = [sentence]
           current_word_count = word_count

if current_chunk:
    chunks.append(' '.join(current_chunk))

Displays cleaned up data for testing

In [6]:
for i, chunk in enumerate(chunks, 1):
    print(f"--- Chunk {i} ---\n{chunk}\n")

--- Chunk 1 ---
Estimates by local consultancy DesRosiers Automotive have the Canadian new light vehicle market up 5. 3% year-on-year in June to 178,000 units. However the year-ago comparison base was particularly low and the SAAR comes in at 1. 81 million, the lowest level so far this year. Q2 sales are up 6. 5% to 532,000 and H1 volumes advance 4. 3% to 958,000. All data below is for Q2. General Motors (+8%) remains the most popular OEM but Hyundai-Kia (+18. 5%) leapfrogs Toyota Motor (+8. 6%) and Ford Motor (-3. 5%) to place #2, a ranking it now also holds over H1 vs. #4 over the Full Year 2024. Honda Canada (+31. 8%) sports the biggest gain in market, with Nissan Group/Mitsubishi (+16. 9%) also in great shape. Stellantis (-9. 7%) underperforms. Looking at brands, Ford (-3. 6%) stays win the lead but Toyota (+6. 5%) catches up and is less than 1,200 sales below over Q2. Hyundai (+13. 8%) passes Chevrolet (+12. 6%) to #3 while Honda (+31. 6%) stays at #5. Kia (+25. 5%), Mazda (+19. 9

Converts the list into a Dictionary

In [7]:
documents = [None] * len(chunks)  # Create an empty list of the correct size

for i in range(len(chunks)):
   documents[i] = Document(page_content=chunks[i])


Adds the dictionaries to vector store

In [8]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f9db2f2d-a020-491b-ae09-27f553a3e9f4',
 'f9de44f9-1e86-4da0-98a6-e654ed667f61',
 '93c34f5f-cc7e-4511-8692-fc0512a6d35e',
 '4254b7d9-b13e-492a-918d-00c582f4ed0e']

Saves the vector store

In [9]:
vector_store.save_local("faiss_index_dir")

Loads the vector store

In [10]:
loaded_vector_store = FAISS.load_local(
    "faiss_index_dir",
    embeddings,  allow_dangerous_deserialization=True
)

In [11]:
retriever = loaded_vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [12]:
template="""You are a helpful assistant. Use the context below to answer the user's question.
Context:
{context}

Question:
{question}

Answer:"""

In [13]:
model_version = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_version)
model = AutoModelForCausalLM.from_pretrained(model_version)

# Create a Hugging Face pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.5
)
# Wrap the pipeline for LangChain compatibility
llm = HuggingFacePipeline(pipeline=pipe)
# Define the Prompt Template
template = """You are a helpful assistant. Use the context below to answer the user's question.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
# Define the RAG Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

Device set to use cpu
C:\Users\hp\AppData\Local\Temp\ipykernel_19384\3758389459.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Function to get answers

In [14]:
def get_answer(question):
    result = rag_chain({"query": question})
    # Extract the generated answer
    return result["result"].split("Answer:")[1].strip()

Langchain integration

In [15]:
st.set_page_config(page_title="Knowledge Chatbot")
st.title("📚 Your Personal Knowledge Chatbot")

if "history" not in st.session_state:
    st.session_state.history = []

if "last_query" not in st.session_state:
    st.session_state.last_query = ""

query = st.text_input("Ask something from your content:")
if query != st.session_state.last_query:
    response = get_answer(query)
    st.session_state.history.append((query, response))
    st.session_state.last_query = query

for q, r in reversed(st.session_state.history):
    st.markdown(f"**You:** {q}")
    st.markdown(f"**Bot:** {r}")

2025-07-25 18:42:57.292 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 18:42:57.293 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 18:42:57.827 
  command:

    streamlit run C:\Users\hp\Desktop\Aykay\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-25 18:42:57.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 18:42:57.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 18:42:57.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 18:42:57.831 Session state does not function when running a script without `streamlit run`